In [1]:
import numpy as np
from sklearn.utils import resample

# Исходные данные
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Параметры бутстрэп
n_iterations = 1000  # количество бутстрэп-выборок
n_size = len(data)    # размер выборки

# Сохраним средние значения для каждой бутстрэп-выборки
means = []

# Процесс бутстрэп
for i in range(n_iterations):
    sample = resample(data, n_samples=n_size)  # выборка с заменой
    mean = np.mean(sample)  # расчет среднего
    means.append(mean)

# Расчет доверительных интервалов
confidence_interval = np.percentile(means, [2.5, 97.5])
print(f'95% доверительный интервал для среднего: {confidence_interval}')

95% доверительный интервал для среднего: [3.8 7.2]


In [47]:
nums = [0,0,1,1,1,2,2,3,3,4]
nums[:] = set(nums)
nums

[0, 1, 2, 3, 4]

In [49]:
nums = [0,0,1,1,1,1,1,2,2,3,3,4]
for i in range(2, len(nums)):
    if nums[i] == nums[i - 2]:
        print(nums[i])

1
1
1
